# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Apr 20 2023 1:50PM,261520,10,Yusen-9398407,Yusen – 3D Matrix,Released
1,Apr 20 2023 1:44PM,261519,10,Clear-9398405,Yusen – 3D Matrix,Released
2,Apr 20 2023 1:35PM,261518,10,9398402,Eywa Pharma Inc.,Released
3,Apr 20 2023 1:16PM,261516,16,9398384,Sartorius Bioprocess Solutions,Released
4,Apr 20 2023 1:15PM,261515,19,2022103472,"GUSA Granules USA, Inc.",Released
5,Apr 20 2023 1:13PM,261514,15,30015873,"Alliance Pharma, Inc.",Released
6,Apr 20 2023 1:13PM,261514,15,30015874,"Alliance Pharma, Inc.",Released
7,Apr 20 2023 12:46PM,261511,10,PRF-58887,Bio-PRF,Released
8,Apr 20 2023 12:46PM,261511,10,PRF-58890,Bio-PRF,Released
9,Apr 20 2023 12:46PM,261511,10,PRF-58894,Bio-PRF,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
20,261515,Released,1
21,261516,Released,1
22,261518,Released,1
23,261519,Released,1
24,261520,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
261515,NaN,NaN,1.0
261516,NaN,NaN,1.0
261518,NaN,NaN,1.0
261519,NaN,NaN,1.0
261520,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
261432,0.0,0.0,1.0
261448,0.0,0.0,5.0
261452,0.0,0.0,1.0
261460,0.0,0.0,1.0
261461,4.0,19.0,4.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
261432,0,0,1
261448,0,0,5
261452,0,0,1
261460,0,0,1
261461,4,19,4


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,261432,0,0,1
1,261448,0,0,5
2,261452,0,0,1
3,261460,0,0,1
4,261461,4,19,4


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,261432,,,1
1,261448,,,5
2,261452,,,1
3,261460,,,1
4,261461,4,19,4


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Apr 20 2023 1:50PM,261520,10,Yusen – 3D Matrix
1,Apr 20 2023 1:44PM,261519,10,Yusen – 3D Matrix
2,Apr 20 2023 1:35PM,261518,10,Eywa Pharma Inc.
3,Apr 20 2023 1:16PM,261516,16,Sartorius Bioprocess Solutions
4,Apr 20 2023 1:15PM,261515,19,"GUSA Granules USA, Inc."
5,Apr 20 2023 1:13PM,261514,15,"Alliance Pharma, Inc."
7,Apr 20 2023 12:46PM,261511,10,Bio-PRF
13,Apr 20 2023 11:35AM,261505,21,"NBTY Global, Inc."
14,Apr 20 2023 11:23AM,261503,15,"Virtus Pharmaceuticals, LLC"
15,Apr 20 2023 11:17AM,261460,15,"Person & Covey, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Apr 20 2023 1:50PM,261520,10,Yusen – 3D Matrix,,,1
1,Apr 20 2023 1:44PM,261519,10,Yusen – 3D Matrix,,,1
2,Apr 20 2023 1:35PM,261518,10,Eywa Pharma Inc.,,,1
3,Apr 20 2023 1:16PM,261516,16,Sartorius Bioprocess Solutions,,,1
4,Apr 20 2023 1:15PM,261515,19,"GUSA Granules USA, Inc.",,,1
5,Apr 20 2023 1:13PM,261514,15,"Alliance Pharma, Inc.",,,2
6,Apr 20 2023 12:46PM,261511,10,Bio-PRF,,,6
7,Apr 20 2023 11:35AM,261505,21,"NBTY Global, Inc.",,,1
8,Apr 20 2023 11:23AM,261503,15,"Virtus Pharmaceuticals, LLC",,,1
9,Apr 20 2023 11:17AM,261460,15,"Person & Covey, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Apr 20 2023 1:50PM,261520,10,Yusen – 3D Matrix,1,,
1,Apr 20 2023 1:44PM,261519,10,Yusen – 3D Matrix,1,,
2,Apr 20 2023 1:35PM,261518,10,Eywa Pharma Inc.,1,,
3,Apr 20 2023 1:16PM,261516,16,Sartorius Bioprocess Solutions,1,,
4,Apr 20 2023 1:15PM,261515,19,"GUSA Granules USA, Inc.",1,,
5,Apr 20 2023 1:13PM,261514,15,"Alliance Pharma, Inc.",2,,
6,Apr 20 2023 12:46PM,261511,10,Bio-PRF,6,,
7,Apr 20 2023 11:35AM,261505,21,"NBTY Global, Inc.",1,,
8,Apr 20 2023 11:23AM,261503,15,"Virtus Pharmaceuticals, LLC",1,,
9,Apr 20 2023 11:17AM,261460,15,"Person & Covey, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Apr 20 2023 1:50PM,261520,10,Yusen – 3D Matrix,1,,
1,Apr 20 2023 1:44PM,261519,10,Yusen – 3D Matrix,1,,
2,Apr 20 2023 1:35PM,261518,10,Eywa Pharma Inc.,1,,
3,Apr 20 2023 1:16PM,261516,16,Sartorius Bioprocess Solutions,1,,
4,Apr 20 2023 1:15PM,261515,19,"GUSA Granules USA, Inc.",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Apr 20 2023 1:50PM,261520,10,Yusen – 3D Matrix,1,NaN,NaN
1,Apr 20 2023 1:44PM,261519,10,Yusen – 3D Matrix,1,NaN,NaN
2,Apr 20 2023 1:35PM,261518,10,Eywa Pharma Inc.,1,NaN,NaN
3,Apr 20 2023 1:16PM,261516,16,Sartorius Bioprocess Solutions,1,NaN,NaN
4,Apr 20 2023 1:15PM,261515,19,"GUSA Granules USA, Inc.",1,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Apr 20 2023 1:50PM,261520,10,Yusen – 3D Matrix,1,0.0,0.0
1,Apr 20 2023 1:44PM,261519,10,Yusen – 3D Matrix,1,0.0,0.0
2,Apr 20 2023 1:35PM,261518,10,Eywa Pharma Inc.,1,0.0,0.0
3,Apr 20 2023 1:16PM,261516,16,Sartorius Bioprocess Solutions,1,0.0,0.0
4,Apr 20 2023 1:15PM,261515,19,"GUSA Granules USA, Inc.",1,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1046068,9,0.0,0.0
15,1568890,11,0.0,0.0
16,261516,1,0.0,0.0
19,2091734,50,43.0,5.0
21,261505,1,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1046068,9,0.0,0.0
1,15,1568890,11,0.0,0.0
2,16,261516,1,0.0,0.0
3,19,2091734,50,43.0,5.0
4,21,261505,1,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,9,0.0,0.0
1,15,11,0.0,0.0
2,16,1,0.0,0.0
3,19,50,43.0,5.0
4,21,1,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,9.0
1,15,Released,11.0
2,16,Released,1.0
3,19,Released,50.0
4,21,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,21
Status,,,,,
Completed,0.0,0.0,0.0,5.0,0.0
Executing,0.0,0.0,0.0,43.0,0.0
Released,9.0,11.0,1.0,50.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,21
0,Completed,0.0,0.0,0.0,5.0,0.0
1,Executing,0.0,0.0,0.0,43.0,0.0
2,Released,9.0,11.0,1.0,50.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,21
0,Completed,0.0,0.0,0.0,5.0,0.0
1,Executing,0.0,0.0,0.0,43.0,0.0
2,Released,9.0,11.0,1.0,50.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()